In [2]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
data_dir="C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_small/fma_small"
# C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_metadata/fma_metadata/echonest.csv
file_path="C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_small/fma_small/000/000002.mp3"

In [6]:

def extract_features(file_path):
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=None)
        
        # Get the Mel-frequency cepstral coefficients
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfccs_processed = np.mean(mfcc.T,axis=0)
        
        # Get the spectral contrast
        stft = np.abs(librosa.stft(y))
        contrast = librosa.feature.spectral_contrast(S=stft, sr=sr)
        contrast_processed = np.mean(contrast.T,axis=0)
        
        # Get the chroma feature
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_processed = np.mean(chroma.T,axis=0)
        
        # Get the mel-scaled spectrogram
        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_processed = np.mean(mel.T,axis=0)

        # Concatenate all features into one array
        features = np.hstack([mfccs_processed, contrast_processed, chroma_processed, mel_processed])
        
    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, error: {e}")
        return None 
    
    return features


In [7]:
def process_all_audio_files(base_directory):
    features_list = []
    track_ids = []

    # Walk through all files in the directory structure
    for subdir, dirs, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.mp3'):
                file_path = os.path.join(subdir, file)
                features = extract_features(file_path)
                if features is not None:
                    features_list.append(features)
                    # Extract track ID from filename
                    track_id = int(file.split('.')[0])
                    track_ids.append(track_id)

    return features_list, track_ids

In [8]:
# Process all audio files and collect features
features, track_ids = process_all_audio_files(data_dir)

C:\Users\annaw\AppData\Local\Temp\ipykernel_79176\538791090.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=None)
C:\Users\annaw\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error encountered while parsing file: C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_small/fma_small\098\098565.mp3, error: 
Error encountered while parsing file: C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_small/fma_small\098\098567.mp3, error: 
Error encountered while parsing file: C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_small/fma_small\098\098569.mp3, error: 
Error encountered while parsing file: C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_small/fma_small\099\099134.mp3, error: 


C:\Users\annaw\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Error encountered while parsing file: C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_small/fma_small\108\108925.mp3, error: 
Error encountered while parsing file: C:/Users/annaw/Python Projects/jupyterNotebooks/DSL Final Project/fma_small/fma_small\133\133297.mp3, error: 


In [10]:
import os
import pandas as pd
from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor

# Initialize Wav2Vec2 processor and feature extractor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")


def extract_wav2vec_features(audio_file):
    audio_input, _ = feature_extractor(audio_file, return_tensors="pt", padding="longest")
    with torch.no_grad():
        features = processor.extract_features(audio_input.input_values)
    return features


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

C:\Users\annaw\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\annaw\.cache\huggingface\hub\models--facebook--wav2vec2-base-960h. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [12]:
# Process all audio files and collect features and labels
# Convert features and labels to DataFrame
features_df = pd.DataFrame(features)
features_df['track_ids'] = track_ids

# Save features to CSV
features_df.to_csv('audio_features.csv', index=False)

In [11]:
import os
import pandas as pd
from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor

# Initialize Wav2Vec2 processor and feature extractor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

def preprocess_audio(file_path):
    # Implement audio preprocessing if required (e.g., resampling)
    # Return the preprocessed audio waveform
    pass

def extract_wav2vec_features(audio_file):
    audio_input, _ = feature_extractor(audio_file, return_tensors="pt", padding="longest")
    with torch.no_grad():
        features = processor.extract_features(audio_input.input_values)
    return features

def process_all_audio_files(base_directory):
    features_list = []
    labels = []

    for subdir, dirs, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.mp3'):
                file_path = os.path.join(subdir, file)
                audio_file = preprocess_audio(file_path)
                if audio_file is not None:
                    features = extract_wav2vec_features(audio_file)
                    features_list.append(features.numpy())
                    labels.append(subdir.split('/')[-1])  # Assuming folder name is the label

    return features_list, labels

# Process all audio files and collect features and labels
features, labels = process_all_audio_files(data_dir)

# Convert features and labels to DataFrame
features_df = pd.DataFrame(features)
features_df['label'] = labels

# Save features to CSV
features_df.to_csv('audio_features.csv', index=False)
